In [1]:
import numpy as np
import pandas as pd
import re
import string

In [4]:
dataset = pd.read_csv('nlp-getting-started/train_Arabic_tweets_positive_20190413.tsv ' , delimiter='\t',header =None , names = ['x' , 'text'])

In [5]:
#Removes Punctuations
def remove_punctuations(data):
    punct_tag=re.compile(r'[^\w\s]')
    data = punct_tag.sub(r'',data)
    return data
    
#Removes HTML syntaxes
def remove_html(data):
    html_tag=re.compile(r'<.*?>')
    data=html_tag.sub(r'',data)
    return data

#Removes URL data
def remove_url(data):
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data

#Removes Emojis
def remove_emoji(data):
    emoji_clean= re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    data=emoji_clean.sub(r'',data)
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data) 
    return data

dataset['text']=dataset['text'].apply(lambda z: remove_html(z))
dataset['text']=dataset['text'].apply(lambda z: remove_punctuations(z))
dataset['text']=dataset['text'].apply(lambda z: remove_url(z))
dataset['text']=dataset['text'].apply(lambda z: remove_emoji(z))

In [6]:
# remove punc
dataset['text'] = dataset.text.apply(lambda x : ''.join([char for char in x if char not in string.punctuation]))

In [7]:
# replace _ by whitespaces 
dataset['text'] = dataset.text.apply(lambda x : x.replace('_' , ' '))

In [8]:
# replace multiple whitespaces by one
dataset['text'] = dataset.text.apply(lambda x : ' '.join(x.split()))

In [9]:
def remove_abb(data):
    abb_clean= re.compile(r"\b[a-zA-Z]\.[a-zA-Z]\b")
    data=abb_clean.sub(r'',data)
    return data
dataset['text'] = dataset['text'].apply(lambda z: remove_abb(z))

In [10]:
dataset['text'] = dataset.text.apply(lambda z : z.replace('pos' , ''))

In [11]:
dataset

,x,text
0,pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...
1,pos,وفي النهاية لن يبقى معك آحدإلا من رأى الجمال ف...
2,pos,من الخير نفسه
3,pos,زلزلالملعبنصرنابيلعب كن عالي الهمه ولا ترضى بغ...
4,pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو المسيا...
...,...,...
22756,pos,السحب الليلة على الايفون رتويت للمرفقة وطبق ال...
22757,pos,لابسة احمر ليه يا ست انتي ايه المناسبة
22758,pos,كلاام جمييل تستاهلمن احبه الله جعل محبته ف قلو...
22759,pos,ألطف صورة ممكن تعبر عن رمضان


In [12]:
class NGramsLM(object):

    def __init__(self , data : pd.Series , n_grams : int):
        self.data = data
        self.n = n_grams
        self.grams_dictionary = self.get_n_grams(self.n)

    def get_n_grams(self , n):

        from collections import defaultdict
        d = defaultdict(int)
        for gram_length in range(1 , n+1):
            for item in self.data:
                item= item.split()
                for i in range(len(item)):
                    d[" ".join(item[i:i+gram_length])] +=1
        return dict((x,y) for (x,y) in sorted(d.items() , key = lambda z : z[1] , reverse= True))

    def get_recommendations(self , text :str , n  : int = 5 ):
        L = [item for item in self.grams_dictionary.keys() if item.startswith(text) and item != text][:n]
        return dict(zip( L ,range(1 , len(L) +1 ) ))
        

In [13]:
ngLM = NGramsLM(dataset.text ,4)

In [19]:
my_text ="صباح "  
ngLM.get_recommendations(my_text , 20)

{'صباح الخير': 1,
 'صباح النور': 2,
 'صباح الورد': 3,
 'صباح الخير حسينعبدالغني': 4,
 'صباح الخير ساعة': 5,
 'صباح الخير ساعة تماما': 6,
 'صباح الفل': 7,
 'صباح الخير لمن': 8,
 'صباح السعادة': 9,
 'صباح النور والسرور': 10,
 'صباح خير': 11,
 'صباح الابتسامة': 12,
 'صباح الخيير': 13,
 'صباح الخيرات': 14,
 'صباح السعاده': 15,
 'صباح الرضا': 16,
 'صباح جميل': 17,
 'صباح المطر': 18,
 'صباح الخيرر': 19,
 'صباح النصر': 20}